In [1]:
from __future__ import print_function
from music21 import *
from collections import Counter, defaultdict
from ea.individual import Measure, Individual, Note
from ea.duration import Duration

import ea.initialisation as initialisation
import ea.modelTrainer as modelTrainer
import ea.musicPlayer as musicPlayer
import ea.simulation as simulation
import ea.fitness as fitness
import collections
import importlib
import music21
import numpy as np
import pandas as pd
import ea.individual as individual
import ea.duration as duration
import ea.constants as constants
import ea.util as util
import random
import nltk
import ea.fitness as fitness
import re
import ea.modelUpdater as modelUpdater


music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [15]:
m = music21.stream.Measure(1)
m.append(music21.note.Note('C4'))
m.append(music21.note.Note('C4'))

m.append(music21.note.Note('C4'))

m[0].duration = music21.duration.Duration(0.25)
len(m)

3

In [ ]:
notes = ['C#4', 'D4', 'E4']
map(lambda x: re.sub('\d+', '', x)

In [20]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)
importlib.reload(modelUpdater)

NameError: name 'ea' is not defined

In [15]:
from ea.individual import Note, Measure, Individual

In [ ]:
len(scores)

## Get corpus

In [2]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
curr_corpus = music21.corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
curr_corpus = lakh_corpus.metadataBundle[0:1006]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [2]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = wiki_corpus.metadataBundle[0:20]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
score = music21.corpus.parse('joplin/maple_leaf_rag.mxl')
scores = util.transpose_piece(score, 'C')
scores = [scores]

In [ ]:
scores[0].show('musicxml')

In [4]:
dur_matrix = modelTrainer.train_duration_matrix(scores)
pitch_matrix = modelTrainer.train_pitch_matrix(scores);

In [ ]:
# Interval matrix
first_notes_per_bar = []
all_intervals = []
for s in scores:
    noteIterator = s.parts[0].getElementsByClass(stream.Measure)
    if len(noteIterator) == 0:
        noteIterator = s.parts[0].notesAndRests.stream()
    for i in noteIterator:
        firstNote = None
        counter = 1
        m = []
        for j in i.notesAndRests:
            if j.isChord:
                curr_note = note.Note(j.root())
            elif j.isRest:
                m.append('REST')
                continue
            else:
                curr_note = j
            if counter == 1:
                if j.isChord:
                    firstNote = note.Note(j.root())
                else:
                    firstNote = j
                pitch_name = firstNote.nameWithOctave
                if '-' in pitch_name or '##' in pitch_name:
                    pitch_name = firstNote.pitch.getEnharmonic().nameWithOctave
                if pitch_name in NOTE_RANGE:
                    first_notes_per_bar.append(pitch_name)
                counter += 1
                continue
            #print(j.beatStr)
            octave = curr_note.octave
            root = note.Note('C')
            root.octave = octave
            interv = interval.Interval(root,  curr_note).name
            m.append(interv)
            counter += 1
            all_intervals.append(interv)
all_intervals

bigrams = list(nltk.bigrams(all_intervals))
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for i1,i2 in bigrams:
    matrix[i1][i2] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
int_matrix = pd.DataFrame(matrix)
int_matrix = int_matrix.fillna(0)


In [ ]:
init_note_vector = defaultdict(lambda: 0)
for i in first_notes_per_bar:
    init_note_vector[i] += 1
total = float(sum(init_note_vector.values()))
for i in init_note_vector:
    init_note_vector[i] /= total
init_note_vector = pd.Series(init_note_vector)

In [7]:
dur_matrix

,eighth,quarter,16th,half
eighth,0.699468,0.371852,0.121091,0.326667
quarter,0.164229,0.417778,0.058540,0.493333
16th,0.104388,0.125926,0.801925,0.033333
half,0.031915,0.084444,0.018444,0.146667


In [8]:
sim = simulation.Simulation(0.5, 100)
population = sim.run_interactively(10, pitch_matrix, dur_matrix)

Starting generation


AttributeError: 'int' object has no attribute 'keys'

In [12]:
sim.run(20, pitch_matrix, dur_matrix)

Starting generation
Initializing population
Starting evolution


TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
sim.pitch_matrix

In [ ]:
chord.Chord(['C3', 'E3', 'G3'], quarterlength='4')

In [32]:
musicPlayer.play(population[0:4])

key = C major


In [ ]:
selection = [population[0], population[1]]
selection

In [ ]:
selected_population_notes = constants.flatten(list(map(lambda x: x.notes, selection)))
selected_population_pitches = list(map(lambda x: x.pitch, constants.flatten(selected_population_notes)))
selected_population_pitches

In [ ]:
population = sim.update(selection=selection)

In [ ]:
[i for sublist in [[[n.pitch for n in m.notes] for m in p.measures] for p in population[0:1]] for i in sublist]

for p in population:
    for m in p.measures:
        for n in p.notes:
            print(n.pitch)

In [ ]:
[population[1].measures[2].notes, list(map(lambda x: x.notes, population[5].measures[1:3]))]

In [ ]:
musicPlayer.play(population[0:1])

In [9]:
selection = population[0:2]
selection[0].measures[0]

Notes: [(B4, quarter, None, None), (C5, half, None, None), (B4, quarter, None, None)], Chord: ['G3', 'B3', 'D3'], Fitness: 0.6666666666666666, User score: NEUTRAL

In [10]:
modelUpdater.update_matrix(selection, pitch_matrix)

,REST,D4,G4,E4,C4,D#4,F4,A4,B4,C#4,C5,D5,E5,G5,D#5,G#4,C#5,F5
REST,0.433437,0.076712,0.060325,0.092784,0.085546,0.0,0.143322,0.060150,0.084211,0.000000,0.009804,0.037736,0.000000,0.125,0.0,0.0,0.0,0.0
D4,0.083591,0.230137,0.039443,0.314433,0.200590,0.0,0.078176,0.067669,0.000000,0.733333,0.019608,0.000000,0.000000,0.000,0.0,0.5,0.0,0.0
E4,0.092879,0.260274,0.148492,0.231959,0.061947,1.0,0.234528,0.007519,0.010526,0.200000,0.068627,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0
C4,0.108359,0.224658,0.023202,0.051546,0.466077,0.0,0.078176,0.045113,0.000000,0.000000,0.019608,0.000000,0.042553,0.000,0.0,0.0,0.0,0.0
G4,0.151703,0.038356,0.327146,0.157216,0.076696,0.0,0.214984,0.406015,0.073684,0.000000,0.107843,0.037736,0.000000,0.000,0.0,0.0,0.0,0.0
F4,0.080495,0.115068,0.232019,0.108247,0.064897,0.0,0.208469,0.060150,0.021053,0.000000,0.009804,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0
C5,0.015480,0.005479,0.027842,0.005155,0.029499,0.0,0.006515,0.060150,0.178947,0.000000,0.156863,0.452830,0.085106,0.000,0.0,0.0,0.0,0.0
B4,0.009288,0.008219,0.032483,0.005155,0.002950,0.0,0.000000,0.082707,0.242105,0.000000,0.343137,0.056604,0.000000,0.000,0.0,0.0,0.0,0.0
A4,0.024768,0.021918,0.081206,0.015464,0.000000,0.0,0.026059,0.195489,0.336842,0.000000,0.068627,0.037736,0.000000,0.000,0.0,0.5,0.0,0.0
C#4,0.000000,0.010959,0.004640,0.012887,0.002950,0.0,0.006515,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0


In [14]:

testIndividual = Individual(
    [
        Measure(
            [
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5))
            ], 0, None),
        Measure(
            [
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5))
            ], 0, None),
        0
    ], 0
)


In [65]:
population[0].measures[0].notes

[(E4, eighth, None, None),
 (E4, eighth, None, None),
 (F4, eighth, None, None),
 (G4, 16th, None, None),
 (G4, 16th, None, None),
 (G4, 16th, None, None),
 (G4, 16th, None, None),
 (A4, 16th, None, None),
 (G4, 16th, None, None),
 (B4, 16th, None, None)]

In [36]:
musicPlayer.play_measure(testIndividual.measures[0])

(C#5, half, None, None)
(C#5, half, None, None)
(C#5, half, None, None)
(C#5, half, None, None)


In [31]:
m = testIndividual.measures[0]

for j in m.notes:
    print(j)

(C#5, half, None, None)
(C#5, half, None, None)
(C#5, half, None, None)
(C#5, half, None, None)


In [24]:
def convert_measure_to_music21_measure(measure: Measure):
    measure.notes: [Note]
    measure = stream.Measure(1)
    for j in measure.notes:
        if j.pitch == 'REST':
            n = note.Rest()
            n.duration = duration.Duration(quarterLength=j.duration.duration_value / 0.25)
        else:
            n = note.Note(j.pitch)
            n.duration = duration.Duration(quarterLength=j.duration.duration_value / 0.25)
        print(j)
        measure.append(n)

    return measure


In [41]:
testIndividual.measures[0].notes[0].duration_value

AttributeError: 'Note' object has no attribute 'duration_value'

In [39]:
for i in range(1):
    print(i)

0
